In [2]:
import pickle
offset_path = "offset.p"
post_pose_opt_path = "pose_opt_qs.p"

In [3]:
with open(post_pose_opt_path, "rb") as file:
    in_dict = pickle.load(file)
    
in_dict

{'kp_data': Array([[[-0.097311  ,  0.2986892 ,  0.02222085, ..., -0.06535974,
           0.36610198,  0.02476426],
         [-0.11202945,  0.30537218,  0.02347705, ..., -0.05853196,
           0.37310544,  0.02471885],
         [-0.13277252,  0.31758168,  0.02126524, ..., -0.0460594 ,
           0.381245  ,  0.02274905],
         ...,
         [-0.12772006,  0.3466903 ,  0.02158621, ..., -0.03661202,
           0.38852534,  0.01684374],
         [-0.12119389,  0.35038656,  0.01963131, ..., -0.03343037,
           0.3899557 ,  0.01673588],
         [-0.11672372,  0.35188216,  0.01751111, ..., -0.03060054,
           0.39065376,  0.01798419]],
 
        [[-0.11440761,  0.3515451 ,  0.01826798, ..., -0.02429817,
           0.39421457,  0.02164671],
         [-0.11299485,  0.35688856,  0.02235741, ..., -0.00572089,
           0.40885192,  0.02603232],
         [-0.10640561,  0.36772895,  0.02469619, ...,  0.00668386,
           0.42265335,  0.02650587],
         ...,
         [-0.06764192,

In [5]:
in_dict["q"].shape

(100, 10, 74)

In [6]:
model = in_dict["mjx_model"]
model

Model(nq=74, nv=73, nu=30, na=30, nbody=66, njnt=68, ngeom=101, nsite=109, nmesh=0, npair=5, nexclude=26, neq=0, nnumeric=0, nM=1119, opt=Option(timestep=Array([0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
       0.002], dtype=float32, weak_type=True), tolerance=Array([1.e-08, 1.e-